In [2]:
from Bio import Entrez

Entrez.email = "brianna.flynn@utexas.edu"  # Required by NCBI

def search_omim_by_gene(gene_symbol):
    try:
        # Search Gene database to get gene ID
        handle = Entrez.esearch(db="gene", term=f"{gene_symbol}[Gene Name] AND Homo sapiens[Organism]")
        record = Entrez.read(handle)
        handle.close()

        if not record['IdList']:
            return f"No NCBI Gene ID found for {gene_symbol}"

        gene_id = record['IdList'][0]

        # Use gene ID to link to OMIM
        link_handle = Entrez.elink(dbfrom="gene", db="omim", id=gene_id)
        link_record = Entrez.read(link_handle)
        link_handle.close()

        omim_ids = [link['Id'] for link in link_record[0]['LinkSetDb'][0]['Link']]
        if not omim_ids:
            return f"No OMIM entries found for {gene_symbol}"

        # Fetch OMIM entry summaries
        summary_handle = Entrez.esummary(db="omim", id=",".join(omim_ids))
        summary_record = Entrez.read(summary_handle)
        summary_handle.close()

        results = []
        for item in summary_record:
            results.append({
                "OMIM ID": item["Id"],
                "Title": item["Title"],
                "Summary": item.get("Summary", "N/A")
            })
        return results

    except Exception as e:
        return f"Error: {e}"

# Example usage
results = search_omim_by_gene("IFT88")
print(results)
for r in results:
    print(f"{r['OMIM ID']}: {r['Title']}\n  Summary: {r['Summary']}\n")


[{'OMIM ID': '600595', 'Title': 'INTRAFLAGELLAR TRANSPORT 88; IFT88', 'Summary': 'N/A'}]
600595: INTRAFLAGELLAR TRANSPORT 88; IFT88
  Summary: N/A



In [4]:
from Bio import Entrez
import xml.etree.ElementTree as ET

Entrez.email = "brianna.flynn@utexas.edu"

def get_omim_description(omim_id):
    handle = Entrez.efetch(db="omim", id=omim_id, retmode="xml")
    record = handle.read()
    handle.close()

    root = ET.fromstring(record)

    descriptions = []
    for text_section in root.findall(".//Section"):
        heading = text_section.findtext("SectionTitle")
        text = text_section.findtext("SectionText")
        if heading and text:
            descriptions.append((heading.strip(), text.strip()))

    print(descriptions)
    return descriptions

# Example usage
descriptions = get_omim_description("600595")  # OMIM ID for IFT88
for heading, text in descriptions:
    print(f"--- {heading} ---\n{text[:500]}...\n")  # Show first 500 chars


[]


In [5]:
import requests

OMIM_API_KEY = "46j5b8deTiuOitXeykLmTg"  # Replace with your actual key

def get_omim_text_sections(mim_number):
    url = "https://api.omim.org/api/entry"
    params = {
        "mimNumber": mim_number,
        "include": "text",  # Get text sections like Clinical Features
        "format": "json",
        "apiKey": OMIM_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "omim" not in data or "entryList" not in data["omim"]:
        return f"No data found for OMIM ID {mim_number}"

    entry = data["omim"]["entryList"][0]["entry"]
    title = entry.get("title", "N/A")

    sections = entry.get("textSectionList", [])
    results = []
    for sec in sections:
        heading = sec["textSection"].get("heading", "Untitled")
        text = sec["textSection"].get("text", "")
        results.append((heading, text.strip()))
    return title, results

# Example usage
title, sections = get_omim_text_sections("600595")  # IFT88
print(f"Title: {title}")
for heading, text in sections:
    print(f"\n--- {heading} ---\n{text[:500]}...\n")


Title: N/A

--- Untitled ---
...


--- Untitled ---
...


--- Untitled ---
...


--- Untitled ---
...


--- Untitled ---
...


--- Untitled ---
...



In [6]:
def get_clinical_synopsis(mim_number):
    url = "https://api.omim.org/api/entry"
    params = {
        "mimNumber": mim_number,
        "include": ["clinicalSynopsis", "text", "geneMap"],
        "format": "json",
        "apiKey": OMIM_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()
    entry = data.get("omim", {}).get("entryList", [{}])[0].get("entry", {})

    title = entry.get("title", "N/A")
    clinical = entry.get("clinicalSynopsis", {})
    text_sections = entry.get("textSectionList", [])

    return title, clinical, text_sections

In [7]:
title, clinical, text_sections = get_clinical_synopsis("600595")  # Example MIM

print(f"Title: {title}\n")
print("=== Clinical Synopsis ===")
if clinical:
    for key, value in clinical.items():
        if isinstance(value, str) and value.strip():
            print(f"{key}: {value}")
else:
    print("No clinical synopsis available.")

print("\n=== Text Sections ===")
for sec in text_sections:
    heading = sec["textSection"].get("heading", "Untitled")
    text = sec["textSection"].get("text", "").strip()
    if text:
        print(f"\n--- {heading} ---\n{text[:500]}...\n")


Title: N/A

=== Clinical Synopsis ===
No clinical synopsis available.

=== Text Sections ===


In [8]:
response = requests.get("https://api.omim.org/api/entry?mimNumber=100820&apiKey=46j5b8deTiuOitXeykLmTg")

In [12]:
import requests
import json

# Use example URL from documentation
url = "https://api.omim.org/api/entry"
params = {
    "mimNumber": "100820",  # Example MIM ID
    "apiKey": OMIM_API_KEY,
    "format": "json"
}

response = requests.get(url, params=params)

# Check if request was successful
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))  # Pretty print the raw response
else:
    print(f"Error {response.status_code}: {response.text}")


{
  "omim": {
    "version": "1.0",
    "entryList": [
      {
        "entry": {
          "prefix": "%",
          "mimNumber": 100820,
          "status": "live",
          "titles": {
            "preferredTitle": "ACHOO SYNDROME",
            "alternativeTitles": "AUTOSOMAL DOMINANT COMPELLING HELIOOPHTHALMIC OUTBURST SYNDROME;;\nPHOTIC SNEEZE REFLEX;;\nSNEEZING FROM LIGHT EXPOSURE;;\nPEROUTKA SNEEZE"
          }
        }
      }
    ]
  }
}


In [17]:
import requests
import json

# Use example URL from documentation
url = "https://api.omim.org/api/entry"
params = {
    "mimNumber": "600595",  # Example MIM ID
    "apiKey": OMIM_API_KEY,
    "format": "json", 
    "include":"all"
}

response = requests.get(url, params=params)

# Check if request was successful
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))  # Pretty print the raw response
else:
    print(f"Error {response.status_code}: {response.text}")


{
  "omim": {
    "version": "1.0",
    "entryList": [
      {
        "entry": {
          "prefix": "*",
          "mimNumber": 600595,
          "status": "live",
          "titles": {
            "preferredTitle": "INTRAFLAGELLAR TRANSPORT 88; IFT88",
            "alternativeTitles": "INTRAFLAGELLAR TRANSPORT 88, CHLAMYDOMONAS, HOMOLOG OF;;\nTG737, MOUSE, HOMOLOG OF; TG737;;\nPOLARIS, MOUSE, HOMOLOG OF;;\nTETRATRICOPEPTIDE REPEAT DOMAIN-CONTAINING PROTEIN 10; TTC10;;\nDAF19, C. ELEGANS, HOMOLOG OF; DAF19;;\nD13S1056E"
          },
          "textSectionList": [
            {
              "textSection": {
                "textSectionName": "cloning",
                "textSectionTitle": "Cloning and Expression",
                "textSectionContent": "{5:Moyer et al. (1994)} cloned and characterized the Ift88 gene, originally called Tg737, which was the site of mutation in a transgenic mouse with an insertional mutation causing a phenotype resembling human autosomal recessive polycys

In [25]:
import requests
import json

OMIM_API_KEY = "46j5b8deTiuOitXeykLmTg"

url = "https://api.omim.org/api/entry"
params = {
    "mimNumber": "600595",  # IFT88
    "apiKey": OMIM_API_KEY,
    "format": "json",
    "include": "all"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    entry = data["omim"]["entryList"][0]["entry"]
    title = entry.get("titles", {}).get("preferredTitle", "N/A")

    cl = entry.get("clinicalSynopsisExists", bool)
    
    if cl:
        print("Clinical Synopsis Exists")
        clinical = entry.get("clinicalSynopsis", "")
        print(clinical)

    else:
        print("No Clinical Synopsis Available")

    
    text_sections = entry.get("textSectionList", [])
    #print(text_sections)

    print(f"\nTitle: {title}")
    print("\n=== Selected Text Sections ===")

    for section in text_sections:
        text_section = section.get("textSection", {})
        heading = text_section.get("textSectionTitle", "")
        print(heading)
        content = text_section.get("text", "").strip()

        if heading in ["Gene Function", "Molecular Genetics"] and content:
            print(f"\n--- {heading} ---\n{content[:1000]}...\n")  # Truncated for readability
else:
    print(f"Error {response.status_code}: {response.text}")


No Clinical Synopsis Available

Title: INTRAFLAGELLAR TRANSPORT 88; IFT88

=== Selected Text Sections ===
Cloning and Expression
Gene Structure
Mapping
Gene Function
Molecular Genetics
Animal Model


In [27]:
import requests
import json

OMIM_API_KEY = "46j5b8deTiuOitXeykLmTg"

url = "https://api.omim.org/api/entry"
params = {
    "mimNumber": "277600",  
    "apiKey": OMIM_API_KEY,
    "format": "json",
    "include": "all"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    entry = data["omim"]["entryList"][0]["entry"]
    title = entry.get("titles", {}).get("preferredTitle", "N/A")

    cl = entry.get("clinicalSynopsisExists", bool)
    
    if cl:
        print("Clinical Synopsis Exists")
        clinical = entry.get("clinicalSynopsis", "")
        print(clinical)

    else:
        print("No Clinical Synopsis Available")

    
    text_sections = entry.get("textSectionList", [])
    #print(text_sections)

    print(f"\nTitle: {title}")
    print("\n=== Selected Text Sections ===")

    for section in text_sections:
        text_section = section.get("textSection", {})
        heading = text_section.get("textSectionTitle", "")
        print(heading)
        content = text_section.get("text", "").strip()

        if heading in ["Gene Function", "Molecular Genetics"] and content:
            print(f"\n--- {heading} ---\n{content[:1000]}...\n")  # Truncated for readability
else:
    print(f"Error {response.status_code}: {response.text}")


Clinical Synopsis Exists
{'inheritance': 'Autosomal recessive {SNOMEDCT:258211005} {UMLS C0441748 HP:0000007} {HPO HP:0000007}', 'growthHeight': 'Short stature, proportionate {SNOMEDCT:772085001} {UMLS C0878660 HP:0003508} {HPO HP:0003508};\nAdult male height 142-169 cm {UMLS C1842215};\nAdult female height 130-157 cm {UMLS C1842216}', 'growthOther': 'Muscular build {UMLS C1842217}', 'headAndNeckHead': 'Brachycephaly {SNOMEDCT:13649004} {UMLS C0221356 HP:0000248} {HPO HP:0000248} {EOM ID:9ccc4d161e106e4d5543286323f007ce IMG:Brachycephaly-Large-small.jpg}', 'headAndNeckFace': 'Maxillary hypoplasia {ICD10CM:M26.02} {ICD9CM:524.03} {UMLS C0240310 HP:0000327} {HPO HP:0000327}', 'headAndNeckEyes': 'Severe myopia {SNOMEDCT:34187009} {UMLS C0271183 HP:0011003} {HPO HP:0011003} {ICD10CM:H52.1} {ICD9CM:367.1};\nGlaucoma (75%) {SNOMEDCT:23986001} {ICD10CM:H40-H42,H40,H40.9} {ICD9CM:365.9,365} {UMLS C0997768,C0017601 HP:0000501} {HPO HP:0000501};\nEctopia lentis (64%) {SNOMEDCT:74969002} {ICD10CM